In [11]:
!pip install -U langchain-community
!pip install langchain_chroma
!pip install tiktoken
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


# Parent Document Retriever
- 父文档检索器

在拆分文档进行检索时，经常会出现相互冲突的需求：

1. 可能想要小文档，以便它们的嵌入能够最准确地反映它们的含义。如果太长，那么嵌入可能会失去意义。
2. 希望拥有足够长的文档以保留每个块的上下文。

ParentDocumentRetriever 通过分割和存储小块数据来实现这种平衡。在检索过程中，它首先获取小块，然后查找这些块的父 ID，并返回那些较大的文档。

注意，“父文档”是指小块源自的文档。这可以是整个原始文档或更大的块。

In [12]:
from langchain.schema import Document
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [16]:
# 绑定Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# 如果不使用Google Colab，该步骤忽略
import os
path = "/content/drive/My Drive/Colab/langchain/RAG/retrieval"
os.chdir(path)

In [23]:
loaders = [
    TextLoader("paul_graham_essay.txt"),
    TextLoader("state_of_the_union.txt"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

# Retrieving full documents
- 检索完整文档

在这种模式下，我们想要检索完整的文档。因此，我们只指定一个child_splitter。

In [24]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [25]:
retriever.add_documents(docs, ids=None)

In [26]:
# 这应该会产生两个键，因为我们添加了两个文档。
list(store.yield_keys())

['16814f60-809f-4b07-ab52-002e420abd92',
 'd65be328-052d-466d-83f2-00525a9a8592']

In [27]:
# 现在调用向量存储搜索功能 - 应该看到它返回小块（因为我们正在存储小块）。
sub_docs = vectorstore.similarity_search("justice breyer")

In [28]:
print(sub_docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


In [29]:
# 现在从整体检索器中检索。这应该返回大文档 - 因为它返回较小块所在的文档。
retrieved_docs = retriever.invoke("justice breyer")

In [30]:
len(retrieved_docs[0].page_content)

38539

# Retrieving larger chunks
- 检索更大的块

有时，完整文档可能太大而无法按原样检索它们。在这种情况下，我们真正想做的是首先将原始文档分割成更大的块，然后将其分割成更小的块。然后我们索引较小的块，但在检索时我们检索较大的块（但仍然不是完整的文档）。



In [31]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()

In [32]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [33]:
retriever.add_documents(docs)

In [34]:
len(list(store.yield_keys()))

66

In [35]:
sub_docs = vectorstore.similarity_search("justice breyer")

In [36]:
print(sub_docs[0].page_content)

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.


In [37]:
retrieved_docs = retriever.invoke("justice breyer")

In [38]:
len(retrieved_docs[0].page_content)

1849

In [39]:
print(retrieved_docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence. 

A former top litigator in private practice. A former federal publi